<a id="introduction"></a>
# Ciencia de Datos Acelerada con GPU
## RAPIDS: Introducción a cuDF
#### Presentador: Mario Inga (BLAZINGSQL)

Notebook adaptado por Tom Drabas (BlazingSQL) de [Introduction to cuDF notebook](https://github.com/rapidsai-community/notebooks-contrib/blob/community_relaunch/getting_started_materials/intro_tutorials_and_guides/02_Introduction_to_cuDF.ipynb) por Paul Hendricks. 

-------

En este Notebook mostraremos cómo usar cuDF DataFrames en RAPIDS.

**Table of Contents**

* [Introduction to cuDF](#introducción)
* [Setup](#configuración)
* [cuDF Series Basics](#series)
* [cuDF DataFrame Basics](#dataframes)
* [Input/Output](#io)
* [cuDF API](#cudfapi)
* [Conclusion](#conclusiones)

<a id="setup"></a>
## Setup

Este notebook fue testeado en Blazing Notebooks, corriendo sobre un GPU de T4 NVIDIA, CON 16GB de RAM: tu sistemas puede que sea diferente y necesites modificar o instalar algún paquete para poder ejecuitar los siguientes ejemplos.

Hemos usado cuDF version 0.17.0.

<a id="series"></a>
## Básicos de cuDF Series

Primero, vamos a cargar la librería de cuDF.

In [75]:
%config IPCompleter.greedy=True

In [1]:
import cudf
print(f'cuDF Version: {cudf.__version__}')

cuDF Version: 0.17.0


Existen dos estructuras de data principales, en cuDF: un objeto `Series` y un objeto `DataFrame`. Múltiples objetos `Series` son usados como columnas por un `DataFrame`. Vamos a explorar la clase `Series` para luego revisar, cómo trabajar con objetos de tipo `DataFrame`.

Entonces, iniciemos!
Podemos crear un objeto `Series` usando la clase `cudf.Series`.

In [2]:
column = cudf.Series([10, 11, 12, 13])
column

0    10
1    11
2    12
3    13
dtype: int64

Podemos observar que `column` es un objeto de tipo `cudf.Series` y que tiene 4 filas.

Otra forma de visualizar `Series` es usando `print`.

In [3]:
print(column)

0    10
1    11
2    12
3    13
dtype: int64


Observamos que nuestro objeto `Series` tiene cuatro filas con valores 10, 11, 12 y 13. Además el tipo de data es `int64`. Hay varias maneras de representar data usando cuDF. Los formatos más comunes son `int8`, `int32`, `int64`, `float32` y `float64`.

Podemos observar además otra columna a la izquierda, con valores 0, 1, 2 y 3. Estos valores representan los index de `Series`. 

In [4]:
print(column.index)

RangeIndex(start=0, stop=4, step=1)


Podemos crear uan nueva columna con index diferentes usando el método `set_index`.

In [5]:
new_column = column.set_index([5, 6, 7, 8]) 
print(new_column)

5    10
6    11
7    12
8    13
dtype: int64


Los index son útiles para operaciones como joins y groupbys.

<a id="dataframes"></a>
## Básicos de cuDF DataFrame

Cómo acabamos de mostrar, cuDF DataFrames son estructuras tabulares de data que residen en el GPU. Interactuamos con estos cuDF DataFrames, de la misma manera en la que interactuamos con Pandas DataFrames que residen en la CPU - con algunas pocas diferencias.

En las siguientes secciones, mostraremos cómo crear y manipular cuDF DataFrames. Para mayor ifnormación en el uso de cuDF DataFrames, pueden revisar la documentación en: https://docs.rapids.ai/api/cudf/stable/

#### Creación de cudf DataFrame usando listas

Hay varias maneras de crear cuDF DataFrame. La más sencilla es instanciar un cuDF DataFrame vacío, y luego usar Python list objects o arreglos NumPy para crear columnas. En la siguiente sentencia, estamos creando un cuDF DataFrame vacío.

In [6]:
df = cudf.DataFrame()
print(df)

Empty DataFrame
Columns: []
Index: []


Luego, podemos crear dos columnas:`key` y `value`, usando el bracket notation con el cuDF DataFrame y almacenando, ya sea una lista de valores de python o un arreglo NumPy en dicha columna. 

In [7]:
import numpy as np; print('NumPy Version:', np.__version__)


# acá creamos dos columnas llamadas "key" y "value"
df['key'] = [0, 1, 2, 3, 4]
df['value'] = np.arange(10, 15)
print(df)

NumPy Version: 1.19.5
   key  value
0    0     10
1    1     11
2    2     12
3    3     13
4    4     14


#### Creación de cudf DataFrame usando una lista de tuples o diccionario

Otra forma de crear un cuDF DataFrame es suministrando un mapping de  de nombres de columna a valores de columna, ya sea con una lista de tuples o usando un diccionario. En los siguientes ejemplos, creamos una lista two-value tuples; el primer valor es el nombre de la columna - por ejemplo, `id` o `timestamp` - y el segundo valor es una lista de objectos Python o arreglos Numpy. Nótese que no tenemos que restringir la data almacenada a tipos comunes de data como integers o floats - podemos usar tipos de data más complejos como datetimes o strings. Vamos a analizar cómo estos tipos de data se comportan en la GPU más adelante.

In [10]:
# Breve ejemplo de arreglos de datos en Python
lista = [1,2,3,4]
lista[0] = 0
print(lista)

tupla = (1,2,3,4)
#tupla[0] = 0
print(tupla)

diccionario = {
    "key1": "value1", 
    "nombre": "Juan",
    "apellido": "Perez"
}
print(diccionario["nombre"])

[0, 2, 3, 4]
(1, 2, 3, 4)
Juan


In [11]:
from datetime import datetime, timedelta

ids = np.arange(5)
t0 = datetime.strptime('2018-10-07 12:00:00', '%Y-%m-%d %H:%M:%S')
timestamps = [(t0+ timedelta(seconds=x)) for x in range(5)]
timestamps_np = np.array(timestamps, dtype='datetime64')

In [12]:
df = cudf.DataFrame()
df['ids'] = ids
df['timestamp'] = timestamps_np
print(df)

   ids           timestamp
0    0 2018-10-07 12:00:00
1    1 2018-10-07 12:00:01
2    2 2018-10-07 12:00:02
3    3 2018-10-07 12:00:03
4    4 2018-10-07 12:00:04


Alternativamente, podemos crear un diccionario de key-value pares, donde cada key en el diccionario repesenta un nombre de columna y cada valor asociado con el key reprsentaría los valores que corresponden en dicha columna.

In [13]:
df = cudf.DataFrame({'id': ids, 'timestamp': timestamps_np})
print(df)

   id           timestamp
0   0 2018-10-07 12:00:00
1   1 2018-10-07 12:00:01
2   2 2018-10-07 12:00:02
3   3 2018-10-07 12:00:03
4   4 2018-10-07 12:00:04


#### Creación de cudf DataFrame desde un Pandas DataFrame

Podemos crear un cuDF DataFrame a partir de un Pandas DataFrame y viceversa.

In [14]:
import pandas as pd
print('Pandas Version:', pd.__version__)


pandas_df = pd.DataFrame({'a': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'b': [0.0, 0.1, 0.2, None, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]})
print(pandas_df)

Pandas Version: 1.1.4
     a    b
0    0  0.0
1    1  0.1
2    2  0.2
3    3  NaN
4    4  0.4
5    5  0.5
6    6  0.6
7    7  0.7
8    8  0.8
9    9  0.9
10  10  1.0


Podemos usar las funciones `cudf.from_pandas` o `cudf.DataFrame.from_pandas` para crear un cuDF DataFrame a partir de un Pandas DataFrame.

In [15]:
df = cudf.from_pandas(pandas_df)
# df = cudf.DataFrame.from_pandas(pandas_df)  # alternative
print(df)

     a     b
0    0   0.0
1    1   0.1
2    2   0.2
3    3  <NA>
4    4   0.4
5    5   0.5
6    6   0.6
7    7   0.7
8    8   0.8
9    9   0.9
10  10   1.0


#### Creación de cuDF DataFrame desde cuDF Series

Podemos crear cuDF DataFrame desde uno o más objetos de cuDF Series, pasando el objeto Series en un diccionario y mapeando cada objeto Series a un nombre de columna.

In [19]:
column1 = cudf.Series([1, 2, 3, 4])
column2 = cudf.Series([5, 6, 7, 8])
column3 = cudf.Series([9, 10, 11, 12])

df = cudf.DataFrame({'a': column1, 'b': column2, 'c': column3})
print(df)

   a  b   c
0  1  5   9
1  2  6  10
2  3  7  11
3  4  8  12


In [20]:
df = cudf.DataFrame({'a': np.arange(0, 100), 'b': np.arange(100, 0, -1)})

In [22]:
df

,a,b
0,0,100
1,1,99
2,2,98
3,3,97
4,4,96
...,...,...
95,95,5
96,96,4
97,97,3
98,98,2


Una segunda forma de inspeccionar un cuDF DataFrame es empaquetar el objeto en la función de Python `print`. Y así podremo ver las filas y columnas del dataframe.

In [23]:
print(df)

     a    b
0    0  100
1    1   99
2    2   98
3    3   97
4    4   96
..  ..  ...
95  95    5
96  96    4
97  97    3
98  98    2
99  99    1

[100 rows x 2 columns]


Para dataframes grandes, preferimos visualizar los primeros pares de filas, para ello podemos usar el método `head` y así visualizar las N filas que queramos.

In [39]:
#df.head()
df.head(10)

,a,b
0,0,100
1,1,99
2,2,98
3,3,97
4,4,96
5,5,95
6,6,94
7,7,93
8,8,92
9,9,91


#### Columnas

cuDF DataFrames almacenan metadata con información sobre columnas o tipo de data. Podemos acceder a las columnas usando el atributo `.columns`.

In [40]:
print(df.columns)

Index(['a', 'b'], dtype='object')


Podemos modificar las columnas de un cuDF DataFrame, modificando el atributo `columns`. Podemos realizar esto configurando el atributo, igual a una lista de strings que representen las nuevas columnas.

In [42]:
df.columns = ['c', 'd']
print(df.columns)
print(df)

Index(['c', 'd'], dtype='object')
     c    d
0    0  100
1    1   99
2    2   98
3    3   97
4    4   96
..  ..  ...
95  95    5
96  96    4
97  97    3
98  98    2
99  99    1

[100 rows x 2 columns]


#### Data Types

Además podemos inspeccionar el tipo de data de las columnas usando el atributo `dtypes`.

In [50]:
#print(df.dtypes)
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   c       100 non-null    float32
 1   d       100 non-null    int32
dtypes: float32(1), int32(1)
memory usage: 800.0 bytes


In [ ]:
df.shape

Podemos modificar el tipo de data de las columnas, pasando en un cuDF Series con los tipos de datos modificados. Atento que algunos errores pueden aparecer for conversiones incoherentes - por ejemplo, cambiar un float a integer o vice versa.

In [53]:
df['c'] = df['c'].astype(np.float32)
df['d'] = df['d'].astype(np.int32)
print(df.dtypes)
#df.info()

c    float32
d      int32
dtype: object


#### Series

cuDF DataFrames están compuestos de filas y columnas. Cada columna está representada usando un objeto de tipo `Series`. Por ejemplo, si dividimos un DataFrame usando una sola columna, retornaremos un objeto de tipo `cudf.dataframe.series.Series`.

In [54]:
print(type(df['c']))
print(df['c'])

<class 'cudf.core.series.Series'>
0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
      ... 
95    95.0
96    96.0
97    97.0
98    98.0
99    99.0
Name: c, Length: 100, dtype: float32


#### Index

Así como los objetos `Series`, cada `DataFrame` tiene un atributo index.

In [55]:
df.index

RangeIndex(start=0, stop=100, step=1)

Podemos usar valores index para subconjuntos de `DataFrame`.

In [61]:
print(df[df.index == 0])
#df.head(1)

     c    d
0  0.0  100


#### Converting a cudf DataFrame to a Pandas DataFrame

Podemos reconvertir un cuDF DataFrame a Pandas DataFrame usando el método `to_pandas`.

In [63]:
pandas_df = df.to_pandas()
print(type(pandas_df))
pandas_df

<class 'pandas.core.frame.DataFrame'>


,c,d
0,0.0,100
1,1.0,99
2,2.0,98
3,3.0,97
4,4.0,96
...,...,...
95,95.0,5
96,96.0,4
97,97.0,3
98,98.0,2


#### Convertir un cudf DataFrame a NumPy Array

Frecuentemente queremos trabajar con arreglos NumPy. Podemos convertir un cuDF DataFrame a un arreglo NumPy, conviertiéndolo primero a un Pandas DataFrame usando el método `to_pandas` y luego accediendo al atributo `values`.

In [65]:
numpy_array = df.to_pandas().values
print(type(numpy_array))
#numpy_array

<class 'numpy.ndarray'>


#### Covertir un cudf DataFrame a cualquier otro formato de Data


Para mayor información, revisa la documentación en: https://docs.rapids.ai/api/cudf/stable/

<a id="io"></a>
## Input/Output

Antes de procesar data y usarla en modelos de machine learning, necesitamos cargarla a la memoria y poder imprimirla después de usarla/editarla. Hay varias formas de hacer esto usando cuDF.

#### Escribir y cargar CSV Files

Podemos usar un cuDF DataFrame directamente de un CSV.

In [66]:
df.to_csv('./dataset.csv', index=False)

Tal vez la forma más común de crear un cuDF DataFrames es cargando una tabla que está almacenada como archivo en el disco. cuDF nos ofrece varias funcionalidades para leer desde diferentes tipos de formatos de data. A continuación, mostraremos lo fácil que es leer un file CSV:

In [70]:
#df_p = pd.read_csv('./dataset.csv')
#print(type(df_p))
#print(df_p)

df = cudf.read_csv('./dataset.csv')
print(type(df))
print(df)

<class 'cudf.core.dataframe.DataFrame'>
       c    d
0    0.0  100
1    1.0   99
2    2.0   98
3    3.0   97
4    4.0   96
..   ...  ...
95  95.0    5
96  96.0    4
97  97.0    3
98  98.0    2
99  99.0    1

[100 rows x 2 columns]


cuDF intenta ser lo más flexible posible, buscando la mayor semejanza a los Pandas API . Para mayor información en parámetros para trabajar con archivos, pueden revisar la documentación de cuDF IO: 

https://rapidsai.github.io/projects/cudf/en/stable/api.html#cudf.io.csv.read_csv

<a id="cudfapi"></a>
## cuDF API

El API de cuDF es bastante simple y similar al Pandas API. En esta sección, vamos a explorar el API de cuDF y mostrar cómo trabajar con operaciones comunes de manipulación de data.

#### Seleccionado Filas o Columnas

Podemos seleccionar filas desde un cuDF DataFrame usando cortas sintaxis. 

In [71]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32)})

In [80]:
print(df[0:5])
#df.head(5)
#print(df[2:5])


     a      b
0  0.0  100.0
1  1.0   99.0
2  2.0   98.0
3  3.0   97.0
4  4.0   96.0


Hay varias maneras de seleccionar una columna desde un cuDF DataFrame.

In [81]:
print(df['a'])
# print(df.a)  # alternative

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
      ... 
95    95.0
96    96.0
97    97.0
98    98.0
99    99.0
Name: a, Length: 100, dtype: float32


Podemos seleccionar múltiples columnas también, usando una lista de nombres de columnas.

In [82]:
print(df[['a', 'b']])

       a      b
0    0.0  100.0
1    1.0   99.0
2    2.0   98.0
3    3.0   97.0
4    4.0   96.0
..   ...    ...
95  95.0    5.0
96  96.0    4.0
97  97.0    3.0
98  98.0    2.0
99  99.0    1.0

[100 rows x 2 columns]


Podemos también seleccionar filasy columnas específicas.

In [85]:
print(df.loc[0:5, ['a']])
# print(df.loc[0:5, ['a', 'b']])  # para seleccionar múltiples columnas, añade múltiples nombres de columnas

     a
0  0.0
1  1.0
2  2.0
3  3.0
4  4.0
5  5.0


#### Definiendo nuevas Columnas


In [87]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})
df

,a,b,c
0,0.0,100.0,100.0
1,1.0,99.0,101.0
2,2.0,98.0,102.0
3,3.0,97.0,103.0
4,4.0,96.0,104.0
...,...,...,...
95,95.0,5.0,195.0
96,96.0,4.0,196.0
97,97.0,3.0,197.0
98,98.0,2.0,198.0


In [97]:
df['d'] = np.arange(200, 300).astype(np.float32)
# df['d'] = np.arange(200, 300).astype(np.float64)

print(df)
print(df.info())

       a      b      c      d
0    0.0  100.0  100.0  200.0
1    1.0   99.0  101.0  201.0
2    2.0   98.0  102.0  202.0
3    3.0   97.0  103.0  203.0
4    4.0   96.0  104.0  204.0
..   ...    ...    ...    ...
95  95.0    5.0  195.0  295.0
96  96.0    4.0  196.0  296.0
97  97.0    3.0  197.0  297.0
98  98.0    2.0  198.0  298.0
99  99.0    1.0  199.0  299.0

[100 rows x 4 columns]
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       100 non-null    float32
 1   b       100 non-null    float32
 2   c       100 non-null    float32
 3   d       100 non-null    float32
dtypes: float32(4)
memory usage: 1.6 KB
None


#### Suprimir Columnas

Podemos querer borrar columnas de nuestro `DataFrame`. Podemos hacerlo usando el método `drop_column`. Vale remarcar que este método borra la columna in-place - lo que significa que el `DataFrame` donde operamos, se verá modificado.

In [107]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})
df

,a,b,c
0,0.0,100.0,100.0
1,1.0,99.0,101.0
2,2.0,98.0,102.0
3,3.0,97.0,103.0
4,4.0,96.0,104.0
...,...,...,...
95,95.0,5.0,195.0
96,96.0,4.0,196.0
97,97.0,3.0,197.0
98,98.0,2.0,198.0


In [108]:
df.drop(columns=['a'], inplace=True)
print(df)

        b      c
0   100.0  100.0
1    99.0  101.0
2    98.0  102.0
3    97.0  103.0
4    96.0  104.0
..    ...    ...
95    5.0  195.0
96    4.0  196.0
97    3.0  197.0
98    2.0  198.0
99    1.0  199.0

[100 rows x 2 columns]


Si queremos borrar una columnas sin modificar el DataFrame original, podemos usar el método `drop`. Este método retornará un nuevo DataFrame sin la columna o columnas eliminadas.

In [109]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})

In [110]:
new_df = df.drop(columns=['a'])

print('Original DataFrame:')
print(df)
print(22 * '-')
print('New DataFrame:')
print(new_df)

Original DataFrame:
       a      b      c
0    0.0  100.0  100.0
1    1.0   99.0  101.0
2    2.0   98.0  102.0
3    3.0   97.0  103.0
4    4.0   96.0  104.0
..   ...    ...    ...
95  95.0    5.0  195.0
96  96.0    4.0  196.0
97  97.0    3.0  197.0
98  98.0    2.0  198.0
99  99.0    1.0  199.0

[100 rows x 3 columns]
----------------------
New DataFrame:
        b      c
0   100.0  100.0
1    99.0  101.0
2    98.0  102.0
3    97.0  103.0
4    96.0  104.0
..    ...    ...
95    5.0  195.0
96    4.0  196.0
97    3.0  197.0
98    2.0  198.0
99    1.0  199.0

[100 rows x 2 columns]


Podemos incluso, pasar una lista de nombres de columna a borrar.

In [111]:
new_df = df.drop(columns=['a', 'b'])

print('Original DataFrame:')
print(df)
print(22 * '-')
print('New DataFrame:')
print(new_df)

Original DataFrame:
       a      b      c
0    0.0  100.0  100.0
1    1.0   99.0  101.0
2    2.0   98.0  102.0
3    3.0   97.0  103.0
4    4.0   96.0  104.0
..   ...    ...    ...
95  95.0    5.0  195.0
96  96.0    4.0  196.0
97  97.0    3.0  197.0
98  98.0    2.0  198.0
99  99.0    1.0  199.0

[100 rows x 3 columns]
----------------------
New DataFrame:
        c
0   100.0
1   101.0
2   102.0
3   103.0
4   104.0
..    ...
95  195.0
96  196.0
97  197.0
98  198.0
99  199.0

[100 rows x 1 columns]


#### Data Incompleta

A veces la data que tenemos no está limpia como quisiéramos - es común tener valores errados o incluso incompletos. cuDF DataFrames puede presentar data incompleta usando el keyword de Python `None`.

In [112]:
df = cudf.DataFrame({'a': [0, None, 2, 3, 4, 5, 6, 7, 8, None, 10],
                     'b': [0.0, 0.1, 0.2, None, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
                     'c': [0.0, 0.1, None, None, 0.4, 0.5, None, 0.7, 0.8, 0.9, 1.0]})
print(df)

       a     b     c
0      0   0.0   0.0
1   <NA>   0.1   0.1
2      2   0.2  <NA>
3      3  <NA>  <NA>
4      4   0.4   0.4
5      5   0.5   0.5
6      6   0.6  <NA>
7      7   0.7   0.7
8      8   0.8   0.8
9   <NA>   0.9   0.9
10    10   1.0   1.0


También podemos completar estos valores incompletos usando otto método: `fillna`. Ambos objetos `Series` y `DataFrame`, implementan este método.

In [113]:
df['c'] = df['c'].fillna(999)
print(df)

       a     b      c
0      0   0.0    0.0
1   <NA>   0.1    0.1
2      2   0.2  999.0
3      3  <NA>  999.0
4      4   0.4    0.4
5      5   0.5    0.5
6      6   0.6  999.0
7      7   0.7    0.7
8      8   0.8    0.8
9   <NA>   0.9    0.9
10    10   1.0    1.0


In [114]:
new_df = df.fillna(-1)
print(new_df)

     a    b      c
0    0  0.0    0.0
1   -1  0.1    0.1
2    2  0.2  999.0
3    3 -1.0  999.0
4    4  0.4    0.4
5    5  0.5    0.5
6    6  0.6  999.0
7    7  0.7    0.7
8    8  0.8    0.8
9   -1  0.9    0.9
10  10  1.0    1.0


#### Boolean Indexing

Al inicio, hemos podido observar cómo podemos seleccionar ciertas filas de nuestro dataset, usando la notación brackets `[]`. Sin embargo, podríamos querer seleccionar filas bajo cierto criterio - esto se llama boolean indexing. Podemos combinar la notación indexing con un arreglo de valores booleanos, para seleccionar ciertas filas que coincidan con este criterio.

In [115]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
df

,a,b,c,d
0,0,0,0,100
1,0,1,1,99
2,0,1,2,98
3,0,0,3,97
4,0,1,4,96
...,...,...,...,...
95,3,1,95,5
96,3,1,96,4
97,3,0,97,3
98,3,0,98,2


In [119]:
mask = df['a'] == 3
# mask = df['a'] == 0
df[mask]

,a,b,c,d
75,3,0,75,25
76,3,1,76,24
77,3,1,77,23
78,3,1,78,22
79,3,1,79,21
80,3,0,80,20
81,3,0,81,19
82,3,1,82,18
83,3,0,83,17
84,3,1,84,16


#### Almacenando Data

La Data frecuentemente no está ordenada antes de trabajar con ella. Ordenar la data es bastante útil para optimizar operaciones como joins y agregaciones, especialmente cuando la data es distribuida.

Podemos ordenar la data en cuDF usando el método `sort_values` y asignando a cada columna el criterio de orden.

In [120]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
print(df.head())

   a  b  c    d
0  0  1  0  100
1  0  1  1   99
2  0  1  2   98
3  0  0  3   97
4  0  0  4   96


In [121]:
print(df.sort_values('d').head())

    a  b   c  d
99  3  0  99  1
98  3  1  98  2
97  3  1  97  3
96  3  1  96  4
95  3  1  95  5


Podemos además especificar si la columna que estamos ordenando, debería ser ordenada de forma ascendente o descendente usando el argumento `ascending` con `True` o `False` según queramos.

In [124]:
#print(df.sort_values('c', ascending=True).head())
print(df.sort_values('c', ascending=False).head())

    a  b   c  d
99  3  0  99  1
98  3  1  98  2
97  3  1  97  3
96  3  1  96  4
95  3  1  95  5


Podemos ordenar múltiples columnas usando una lista de nombres de columnas.

In [125]:
print(df.sort_values(['a', 'b']).head())

    a  b   c   d
3   0  0   3  97
4   0  0   4  96
7   0  0   7  93
8   0  0   8  92
10  0  0  10  90


Podemos además ser específicos con aquellas columnas que deben ser ordenadas, de forma ascendente o descendente, usando una lista de valores booleanos, donde cada uno mapea una columna respectivamente.

In [126]:
print('Sort with all columns specified descending:')
print(df.sort_values(['a', 'b'], ascending=False).head())
print(44 * '-')
print('Sort with both a descending and b ascending:')
print(df.sort_values(['a', 'b'], ascending=[False, True]).head())

Sort with all columns specified descending:
    a  b   c   d
75  3  1  75  25
78  3  1  78  22
80  3  1  80  20
82  3  1  82  18
87  3  1  87  13
--------------------------------------------
Sort with both a descending and b ascending:
    a  b   c   d
76  3  0  76  24
77  3  0  77  23
79  3  0  79  21
81  3  0  81  19
83  3  0  83  17


/opt/conda-environments/rapids-stable/lib/python3.8/site-packages/cudf/core/frame.py:2560: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  warnings.warn(


#### Operaciones estadísticas

Existen varias operaciones estadísticas que podemos usar. Estas operaciones pueden ser aplicadas a ambos: `Series` y `DataFrame`.

In [127]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
df

,a,b,c,d
0,0,1,0,100
1,0,0,1,99
2,0,0,2,98
3,0,1,3,97
4,0,1,4,96
...,...,...,...,...
95,3,0,95,5
96,3,0,96,4
97,3,1,97,3
98,3,1,98,2


In [129]:
df['a'].sum()

150

In [135]:
df.sum()


a     150
b      47
c    4950
d    5050
dtype: int32

In [139]:
df.describe()

,a,b,c,d
count,100.000000,100.000000,100.000000,100.000000
mean,1.500000,0.470000,49.500000,50.500000
std,1.123666,0.501614,29.011492,29.011492
min,0.000000,0.000000,0.000000,1.000000
25%,0.750000,0.000000,24.750000,25.750000
50%,1.500000,0.000000,49.500000,50.500000
75%,2.250000,1.000000,74.250000,75.250000
max,3.000000,1.000000,99.000000,100.000000


#### Operaciones Applymap

Si bien cuDF nos permite definir nuevas columnas, es normal querer trabajar con funciones más complejas. Podemos definir una función y usar el método `applymap` para aplicar la función a cada valor en una columna. Vayamos a un ejemplo simple, el cual puede fácilmente aplicarse a workfloks más complejos.

In [140]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
df

,a,b,c,d
0,0,1,0,100
1,0,0,1,99
2,0,1,2,98
3,0,0,3,97
4,0,0,4,96
...,...,...,...,...
95,3,0,95,5
96,3,0,96,4
97,3,1,97,3
98,3,1,98,2


In [141]:
def add_ten_to_x(x):
    return x + 10

print(df['c'].applymap(add_ten_to_x))

0      10
1      11
2      12
3      13
4      14
     ... 
95    105
96    106
97    107
98    108
99    109
Name: c, Length: 100, dtype: int64


#### Histogramming

Podemos acceder al conteo de valores de una columna usando el método `value_counts`. Ojo, que este método es usado mayormente en columnas con data discreta, por ejemplo integers, strings, categoricals, etc. Puede que no estemos tan interesados en el conteo de valores de data numérica, por ejemplo qué tan seguido aparece el valor 2.1. El resultado del método `value_counts` puede ser usado con librerías de Python plotting como Matplotlib o Seaborn para generar visualizaciones como histogramas.

In [142]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
df.head()

,a,b,c,d
0,0,1,0,100
1,0,1,1,99
2,0,1,2,98
3,0,0,3,97
4,0,0,4,96


In [146]:
result = df['a'].value_counts()
#result = df['b'].value_counts()
#result = df['c'].value_counts()
print(result)

0    25
1    25
2    25
3    25
Name: a, dtype: int32


#### Concatenaciones

Todo data scientists suele trabajar con múltiples fuentes de data, y lo ideal en pode combinar en una sola representación meas robusta y completa. Estas operaciones son llamadas concatenaciones y joins. Podemos concatenar dos o más dataframes juntos por filas o por columnas, usando una lista de los dataframes a la función `cudf.concat` y especificando el eje de concantenación para los dataframes.

Si queremos concantenar dataframes por filas, podemos especificar `axis=0`. Para concantenar por columnas, podemos especificar`axis=1`.

In [147]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})

In [148]:
df = cudf.concat([df1, df2], axis=0)
df

,a,b,c,d
0,0,0,0,100
1,0,1,1,99
2,0,0,2,98
3,0,1,3,97
4,0,0,4,96
...,...,...,...,...
95,3,0,95,5
96,3,0,96,4
97,3,0,97,3
98,3,1,98,2


In [149]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'e': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'f': np.random.randint(2, size=100).astype(np.int32), 
                      'g': np.arange(0, 100).astype(np.int32), 
                      'h': np.arange(100, 0, -1).astype(np.int32)})

In [150]:
df = cudf.concat([df1, df2], axis=1)
df

,a,b,c,d,e,f,g,h
0,0,1,0,100,0,0,0,100
1,0,0,1,99,0,1,1,99
2,0,1,2,98,0,1,2,98
3,0,0,3,97,0,1,3,97
4,0,0,4,96,0,0,4,96
...,...,...,...,...,...,...,...,...
95,3,0,95,5,3,1,95,5
96,3,1,96,4,3,1,96,4
97,3,1,97,3,3,0,97,3
98,3,1,98,2,3,1,98,2


#### Joins / Merges

A múltiples dataframes se les puede aplicar un join, usando una (o múltiples) columnas(s). Hay dos sintaxis para ejecutar estos joins:

* El método `DataFrame.merge` y pasar a otro dataframe para unir, o
* La función `cudf.merge` y usar uno de los dataframe para la unión.

Ambas sintaxis pueden usar una lista de nombres de columna a un argumento adicional con el keyword `on` - esto especificara qué columnas del dataframe serán unidas. Si este keyword no es especificado, cuDF por default aplicará el join usando los nombres de columnas que aparezcan en ambos dataframes.

In [151]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'e': np.arange(0, 100).astype(np.int32), 
                      'f': np.arange(100, 0, -1).astype(np.int32)})

In [152]:
df = df1.merge(df2)
print(df.head())

   a  b   c   d   e   f
0  1  1  32  68  25  75
1  1  0  33  67  28  72
2  1  1  34  66  25  75
3  1  0  35  65  28  72
4  1  1  36  64  25  75


In [153]:
df = df1.merge(df2, on=['a'])
print(df.head())

   a  b_x  c    d  b_y  e    f
0  0    0  0  100    1  0  100
1  0    0  1   99    1  0  100
2  0    1  2   98    1  0  100
3  0    0  3   97    1  0  100
4  0    1  4   96    1  0  100


In [154]:
df = df1.merge(df2, on=['a', 'b'])
print(df.head())

   a  b  c    d  e    f
0  0  0  0  100  1   99
1  0  0  1   99  1   99
2  0  1  2   98  0  100
3  0  0  3   97  1   99
4  0  1  4   96  0  100


In [155]:
df = cudf.merge(df1, df2)
print(df.head())

   a  b  c    d  e    f
0  0  0  0  100  1   99
1  0  0  1   99  1   99
2  0  1  2   98  0  100
3  0  0  3   97  1   99
4  0  1  4   96  0  100


In [156]:
df = cudf.merge(df1, df2, on=['a'])
print(df.head())

   a  b_x  c    d  b_y  e    f
0  0    0  0  100    1  0  100
1  0    0  1   99    1  0  100
2  0    1  2   98    1  0  100
3  0    0  3   97    1  0  100
4  0    1  4   96    1  0  100


In [157]:
df = cudf.merge(df1, df2, on=['a', 'b'])
print(df.head())

   a  b  c    d  e    f
0  0  0  0  100  1   99
1  0  0  1   99  1   99
2  0  1  2   98  0  100
3  0  0  3   97  1   99
4  0  1  4   96  0  100


#### Groupbys

Una operación muy usada cuando se trabaja con datasets es el groupby. Agrupar la data usando llaves y agregando valores mapeados a esas llaves. Por ejemplo, podríamos querer agregar múltiples medidas de temperatura tomadas durante el día de un sensor específico y promediar dichas temperaturas para encontrar el promedio diario de un lugar.

cuDF nos permite realizar la anterior operación usando el método `groupby`. Esto creará un objeto de tipo `cudf.groupby.groupby.Groupby` donde podremos operar usando funciones de agregación como `sum`, `var`, o más complejas definidas por el usuario.

Podemos también especificar múltiples columnas a un grupo al usar una lista de nombres de columnas con el método `groupby`.

In [158]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
print(df.head())

   a  b  c    d
0  0  0  0  100
1  0  0  1   99
2  0  1  2   98
3  0  1  3   97
4  0  1  4   96


In [159]:
grouped_df = df.groupby('a')
grouped_df

In [160]:
aggregation = grouped_df.sum()
print(aggregation)

    b     c     d
a                
0  12   300  2200
1  13   925  1575
2  15  1550   950
3  12  2175   325


In [161]:
aggregation = df.groupby(['a', 'b']).sum().to_pandas()
print(aggregation)

        c     d
a b            
0 0   160  1140
  1   140  1060
1 0   449   751
  1   476   824
2 0   625   375
  1   925   575
3 0  1134   166
  1  1041   159


#### One Hot Encoding

Data scientists suelen trabajar con data discreta como integers o categorías. Sin embargo, esta data puede ser representada usando el formato One Hot Encoding.

cuDF nos permite convertir la data discreta en formato One Hot Encoding usando el método `one_hot_encoding`. Podemos pasar este método a un nombre de columna para la conversión, un prefijo con el cual anteponer cada nueva colmuna, y las categorías de data para crear las nuevas columnas. Podemos usar en todas las categorías de la data discreta o subcategoría - cuDF manejará de forma flexible ambas y sólo creará nuevas columnas para las categorías específicas.

In [162]:
categories = [0, 1, 2, 3]
df = cudf.DataFrame({'a': np.repeat(categories, 25).astype(np.int32), 
                     'b': np.arange(0, 100).astype(np.int32), 
                     'c': np.arange(100, 0, -1).astype(np.int32)})
print(df.head())

   a  b    c
0  0  0  100
1  0  1   99
2  0  2   98
3  0  3   97
4  0  4   96


In [163]:
result = df.one_hot_encoding('a', prefix='a_', cats=categories)
print(result.head())
print(result.tail())

   a  b    c  a__0  a__1  a__2  a__3
0  0  0  100   1.0   0.0   0.0   0.0
1  0  1   99   1.0   0.0   0.0   0.0
2  0  2   98   1.0   0.0   0.0   0.0
3  0  3   97   1.0   0.0   0.0   0.0
4  0  4   96   1.0   0.0   0.0   0.0
    a   b  c  a__0  a__1  a__2  a__3
95  3  95  5   0.0   0.0   0.0   1.0
96  3  96  4   0.0   0.0   0.0   1.0
97  3  97  3   0.0   0.0   0.0   1.0
98  3  98  2   0.0   0.0   0.0   1.0
99  3  99  1   0.0   0.0   0.0   1.0


In [167]:
result = df.one_hot_encoding('a', prefix='a_', cats=[0, 1, 2])
# result = df.one_hot_encoding('a', prefix='a_', cats=[1, 2])
print(result.head())
print(result.tail())

   a  b    c  a__0  a__1  a__2
0  0  0  100   1.0   0.0   0.0
1  0  1   99   1.0   0.0   0.0
2  0  2   98   1.0   0.0   0.0
3  0  3   97   1.0   0.0   0.0
4  0  4   96   1.0   0.0   0.0
    a   b  c  a__0  a__1  a__2
95  3  95  5   0.0   0.0   0.0
96  3  96  4   0.0   0.0   0.0
97  3  97  3   0.0   0.0   0.0
98  3  98  2   0.0   0.0   0.0
99  3  99  1   0.0   0.0   0.0


<a id="conclusion"></a>
## Conclusion

In this notebook, we showed how to work with cuDF DataFrames in RAPIDS.

Para aprender más de RAPIDS, puedes revisar los siguientes links: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)